In [7]:
import polars as pl
import pandas as pd

In [8]:
df = pd.read_csv('/home/sebacastillo/willow/output/news_narcotráfico_related_2023-08-09_1816.csv')

In [9]:
df.head()

,date_extract,date_article,content,link,authors
0,2023-08-09 18:15:38,2023-08-09T00:00:00.000000,"9 Ago, 2023 Por María Laura Balonga Fuentes j...",https://www.infobae.com/sociedad/policiales/20...,María Laura Balonga
1,2023-08-09 18:15:38,2023-08-09T00:00:00.000000,"9 Ago, 2023 Los dos hermanos que fueron deteni...",https://www.infobae.com/sociedad/policiales/20...,n-a
2,2023-08-09 18:15:38,2023-08-09T00:00:00.000000,"9 Ago, 2023 La mamá de Morena Domínguez —la ne...",https://www.infobae.com/sociedad/policiales/20...,n-a
3,2023-08-09 18:15:38,2023-08-09T00:00:00.000000,"9 Ago, 2023 Por Walter Vazquez Vecinos de Lan...",https://www.infobae.com/sociedad/policiales/20...,Walter Vazquez
4,2023-08-09 18:15:38,2023-08-09T00:00:00.000000,"9 Ago, 2023 Por Eduardo Menegazzi En un hote...",https://www.infobae.com/politica/2023/08/09/sc...,Eduardo Menegazzi


In [12]:
df['date_extract'] = pd.to_datetime(df['date_extract'])
df['date_article'] = pd.to_datetime(df['date_article'])
print(df.head(1))

         date_extract date_article  \
0 2023-08-09 18:15:38   2023-08-09   

                                             content  \
0  9 Ago, 2023 Por  María Laura Balonga Fuentes j...   

                                                link              authors  
0  https://www.infobae.com/sociedad/policiales/20...  María Laura Balonga  
